# Pauli Objects

## Introduction and Arithmetic

Pauli operators acting on the $N$-qubit Hilbert space are ubiquitous in quantum circuit computations. In domain applications like QAOA and VQE, these often appear with a sparse structure corresponding to the physics of the problem at hand. To efficiently manipulate such operators and compute expectation values over the relevant Pauli strings, `quasar` features a `Pauli` class (and associated utility classes) that provide for sparse representation and intuitive manipulation of Pauli-type objects. 

To get started, let's grab some utility objects that generate the Pauli operators on individual qubit indices:

In [ ]:
import numpy as np
import quasar
I, X, Y, Z = quasar.Pauli.IXYZ()

Now, we can use these to start building up Pauli operators. 

In [ ]:
pauli = I[-1] + X[0] - X[0] * Y[1] + 0.5 * X[0] * Y[1] * Z[2] - X[0] * Y[1] * Z[2] / 4.0
pauli += 1.0
pauli += 2.0 * (X[0] + Z[0]) * (X[0] + Y[0])
pauli += pauli
pauli /= 2.0
print(pauli)

How did that work? Next, `I/X/Y/Z` are "`PauliStarter`" objects that generate `Pauli` objects when indexed by the `[]` operator:

In [ ]:
print(type(X))
print(X[3])
print(type(X[3]))

**Technical Note:** Note that in the `Pauli` objects themselves, we only store the overall `I` operator and products of `X`, `Y`, and `Z` strings. Therefore, the `I` `PauliStarter` object is indexed the same as the `X`, `Y`, and `Z` `PauliStarter` objects, but the specific index applied to the `I` `PauliStarter` object has no effect. A loose (but not required convention) is to use `I[-1]` to indicate the overall `I` operator:

In [ ]:
print(I[-1])
print(I[1])
print(I[-1] * X[0])
print(I[1] * X[0])

Next, we have overloaded all relevant arithmetic operators within the `Pauli` class to allow arithmetic composition of linear combinations of products of single-qubit Pauli operators. These work with either `float` or `complex` coefficients:

In [ ]:
print(-1.j * X[0])

Note that products of Pauli operators on the same qubit index are automatically reduced to linear or constant complexity by the Pauli multiplication rules:

In [ ]:
print(X[0]*X[0])
print(X[0]*Y[0])
print(X[0]*Z[0])
print(Y[0]*X[0])
print(Y[0]*Y[0])
print(Y[0]*Z[0])
print(Z[0]*X[0])
print(Z[0]*Y[0])
print(Z[0]*Z[0])
print(X[0]*Y[0]*Z[0])
print(Y[0]*X[0]*Z[0])

This is primarily useful for performing derivations. 

## Pauli Summary Attributes

The unique occupied qubit indices over all strings encountered in a `Pauli` object are provided in the `qubits` property:

In [ ]:
pauli = X[0] + Y[2] + Z[3]
print(pauli.qubits)

The attribute `nqubit` is the total number of qubits between `min_qubit` and `max_qubit` (including any blank qubit indices). The attribute `nqubit_sparse` is the number of occupied qubits (excluding bland qubit indices):

In [ ]:
print(pauli.nqubit)
print(pauli.nqubit_sparse)
print(pauli.min_qubit)
print(pauli.max_qubit)

The maximum order attribute is the maximum number of one-qubit Pauli operators currently present in the `Pauli` object:

In [ ]:
print(pauli.max_order)

The number of terms attribute is the number of Pauli strings currently present in the `Pauli` object:

In [ ]:
print(pauli.nterm)

These are easily printable in a handy summary string:

In [ ]:
print(pauli.summary_str)

## Dictionary Structure

The `Pauli` class actually inherits from the `collections.OrderedDict` class, and internally is represented as an `OrderedDict` of `PauliString : float/complex` pairs.

In [ ]:
pauli = X[0] + X[0] * Y[1]
for string, value in pauli.items():
    print('%-8s : %6s' % (string, value))

We can use `dict` access methods to access and modify the coefficients of existing Pauli strings:

In [ ]:
pauli[quasar.PauliString.from_string('X0*Y1')] += 1.0
print(pauli)

and even add new Pauli strings:

In [ ]:
pauli[quasar.PauliString.from_string('X0*Y4')] = 1.0
print(pauli)

All `dict` access methods have been overloaded to also accept `str` arguments, which will be converted to `PauliString` under the hood:

In [ ]:
pauli['X0'] += 1.0
print(pauli['X0'])
print(pauli['X0*Y1'])

## PauliString and PauliOperator Utility Classes

Utility class `PauliString` represents a string of one or more `PauliOperator` objects:

In [ ]:
string = quasar.PauliString.from_string('X0*Y1*Z2')
print(string)
print(string.order)
print(string.qubits)
print(string.chars)

From a data structure perspective, `PauliString` inherits from `tuple` and contains a `tuple` of `PauliOperator` objects:

In [ ]:
print(string[0])
print(type(string[0]))

Utility class `PauliOperator` represents a one-qubit Pauli operator with a given character and qubit index:

In [ ]:
operator = quasar.PauliOperator.from_string('Y10')
print(operator)
print(operator.qubit)
print(operator.char)

## Symmetry Considerations

In practical problems involving two-qubit and $k$-qubit interactions, one usually encounters one of two possible conventions in the literature/code structure: the upper triangular representation or the symmetric representation. To permit both possibilities, `quasar` considers, e.g., `Z0*Z1` and `Z1*Z0` to be inequivalent `PauliString` objects:

In [ ]:
print(Z[0]*Z[1] + Z[1]*Z[0])

In [ ]:
print(Z[0]*Z[1] + Z[0]*Z[1])

## Additional Math Functions

A few additional math functions are provided in `Pauli` for convenience/advanced users. The `conj` property returns a new version of the `Pauli` object with the values conjugated:

In [ ]:
print(pauli.conj)

The `dot` method takes the dot product of coincident Pauli strings in two `Pauli` objects:

In [ ]:
pauli = 3.0 * I[-1] + 4.0 * Z[0] * Z[1]
print(pauli.dot(pauli))

The `norm2` property returns the square root of the dot product of `self` with `self`:

In [ ]:
print(pauli.norm2)

The `norminf` property returns the magnitude of the largest coefficient of `self`:

In [ ]:
print(pauli.norminf)

The `zero` static method returns a new `Pauli` object initialized with no strings:

In [ ]:
print(quasar.Pauli.zero())

The `zeros_like` static method returns a new `Pauli` object initialized with the strings of another `Pauli` object, but with coefficient values initialized to zero:

In [ ]:
print(quasar.Pauli.zeros_like(pauli))

Many times, arithmetic manipulations yield Pauli strings with small or even zero coefficient values. A new `Pauli` object with such strings removed can be obtained by calling the `sieved` method, which takes an optional `cutoff` argument:

In [ ]:
pauli -= 3.0 * I[-1]
print(pauli)

In [ ]:
print(pauli.sieved())

In [ ]:
print(pauli.sieved(cutoff=1.0E-14))

## Hilbert Space Representation

Sometimes, it can be quite useful to see what a `Pauli` object looks like in the computational-basis Hilbert space, or to see how the `Pauli` object acts on a statevector in the computational-basis Hilbert space. 

In [ ]:
pauli = X[0] * X[1]
print(pauli.to_matrix())
print(pauli.matrix_vector_product(np.array([1, 2, 3, 4])))

It can also be very useful to convert from a computational-basis Hilbert space operator to an equivalent `Pauli` form:

In [ ]:
matrix = quasar.Matrix.YY
print(matrix)
pauli = quasar.Pauli.from_matrix(matrix)
print(pauli)